## 新华社文本抄袭检测<br>
- 项目介绍：
基于TF-IDF的方法抽取文本特征，判断一篇不是新华的文章，是否涉嫌抄袭新华社的文章。

### 读取数据

In [1]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv('data.csv',encoding='gb18030')

In [6]:
df.head()

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...


- 注意到content有不少缺失值

In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89611 entries, 0 to 89610
Data columns (total 8 columns):
id         89611 non-null int64
author     79396 non-null object
source     89609 non-null object
content    87054 non-null object
feature    89611 non-null object
title      89577 non-null object
url        87144 non-null object
y          89611 non-null float64
dtypes: float64(1), int64(1), object(6)
memory usage: 5.5+ MB


- 处理方法：把缺失样本直接扔掉

In [90]:
new_df = df.dropna(subset=['content'])

In [91]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87054 entries, 0 to 89610
Data columns (total 8 columns):
id         87054 non-null int64
author     76925 non-null object
source     87052 non-null object
content    87054 non-null object
feature    87054 non-null object
title      87054 non-null object
url        87054 non-null object
y          87054 non-null float64
dtypes: float64(1), int64(1), object(6)
memory usage: 6.0+ MB


In [92]:
target = new_df[new_df['source']=='新华社']
target.head()

,id,author,source,content,feature,title,url,y
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....,1.0
39,89578,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态,http://world.huanqiu.com/hot/2017-06/10866229....,1.0
95,89522,NaN,新华社,新华社韩国济州6月18日电综述：亚投行第二届年会三大亮点\r\n新华社记者 耿学鹏 严蕾\r...,"{""type"":""国内新闻"",""site"":""环球"",""commentNum"":""0"",""j...",综述：亚投行第二届年会三大亮点,http://china.huanqiu.com/hot/2017-06/10866439....,1.0
113,89504,NaN,新华社,新华社北京6月18日电 经军委领导批准，《军营理论热点怎么看·2017》日前印发全军。\r\...,"{""type"":""国内新闻"",""site"":""环球"",""commentNum"":""0"",""j...",《军营理论热点怎么看·2017》印发全军,http://china.huanqiu.com/hot/2017-06/10866447....,1.0
305,89312,NaN,新华社,新华社兰州6月19日电（记者张钦）记者19日了解到，刚刚出台的《甘肃省网络扶贫行动的实施方案...,"{""type"":""国内新闻"",""site"":""环球"",""commentNum"":""0"",""j...",甘肃力推网络扶贫：贫困山区到2020年实现4G网络全覆盖,http://china.huanqiu.com/hot/2017-06/10866626....,1.0


- 注意，新华社的文章占文章总数的90%以上！
- 分类器即使把所有文章都归为正例，准确率也会高达90%
- 我们要找到一个准确率远高于90%的分类器才行！

In [93]:
len(target)/len(new_df)

0.9035885772049532

### 处理数据

- 属于新华社的文本为1，非新华社文本为0

In [94]:
new_df.loc[new_df['source'] == '新华社','y'] = 1
new_df.loc[new_df['source'] != '新华社','y'] = 0
data = new_df[['content','y']]

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [95]:
data.head()

,content,y
0,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,0.0
1,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,0.0
2,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,0.0
3,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,1.0
4,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,0.0


In [96]:
data['y'].value_counts()

1.0    78661
0.0     8393
Name: y, dtype: int64

- DataFrame转List，为后续jieba分词作准备

In [97]:
content = data['content'].tolist()
y = data['y'].values

In [105]:
len(content),len(y)

(87054, 87054)

In [100]:
content[:3]

['此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/体验版内测，稳定版暂不受影响），以确保工程师可以集中全部精力进行系统优化工作。有人猜测这也是将精力主要用到MIUI 9的研发之中。\r\nMIUI 8去年5月发布，距今已有一年有余，也是时候更新换代了。\r\n当然，关于MIUI 9的确切信息，我们还是等待官方消息。\r\n',
 '骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考虑性能而去屏蔽掉小核心。相反，他们正联手微软，找到一种适合桌面平台的、兼顾性能和功耗的完美方案。\r\n报道称，微软已经拿到了一些新的源码，以便Windows 10更好地理解big.little架构。\r\n资料显示，骁龙835作为一款集成了CPU、GPU、基带、蓝牙/Wi-Fi的SoC，比传统的Wintel方案可以节省至少30%的PCB空间。\r\n按计划，今年Q4，华硕、惠普、联想将首发骁龙835 Win10电脑，预计均是二合一形态的产品。\r\n当然，高通骁龙只是个开始，未来也许还能见到三星Exynos、联发科、华为麒麟、小米澎湃等进入Windows 10桌面平台。\r\n',
 '此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n至于电池缩水，可能与刘作虎所说，一加手机5要做市面最轻薄大屏旗舰的设定有关。\r\n按照目前掌握的资料，一加手机5拥有5.5寸1080P三星AMOLED显示屏、6G/8GB RAM，64GB/128GB ROM，双1600万摄像头，备货量“惊喜”。\r\n根据京东泄露的信息，一加5起售价是xx99元，应该是在2799/2899/2999中的某个。\r\n']

In [32]:
import jieba
import re

- 把文本冗余字符去除，然后进行jieba分词处理，再放入列表中做成语料库
- TF-IDF处理corpus，需要corpus是已经分好词的句子的集合

In [78]:
def clean(text):
    text = ''.join(re.findall(r'\w+',text))
    new_text = ' '.join(jieba.cut(text))
    return new_text

In [101]:
corpus = []
for i in range(len(content)):
    new_content = clean(content[i])
    corpus.append(new_content)

- 耗时良久，分词成功！

In [102]:
corpus[:3]

['此外 自 本周 6 月 12 日起 除 小米 手机 6 等 15 款 机型 外 其余 机型 已 暂停 更新 发布 含 开发 版 体验版 内测 稳定版 暂不受 影响 以 确保 工程师 可以 集中 全部 精力 进行 系统优化 工作 有人 猜测 这 也 是 将 精力 主要 用到 MIUI9 的 研发 之中 MIUI8 去年 5 月 发布 距今已有 一年 有余 也 是 时候 更新换代 了 当然 关于 MIUI9 的 确切 信息 我们 还是 等待 官方消息',
 '骁龙 835 作为 唯一 通过 Windows10 桌面 平台 认证 的 ARM 处理器 高通 强调 不会 因为 只 考虑 性能 而 去 屏蔽掉 小 核心 相反 他们 正 联手 微软 找到 一种 适合 桌面 平台 的 兼顾 性能 和 功耗 的 完美 方案 报道 称 微软 已经 拿到 了 一些 新 的 源码 以便 Windows10 更好 地 理解 biglittle 架构 资料 显示 骁龙 835 作为 一款 集成 了 CPUGPU 基带 蓝牙 WiFi 的 SoC 比 传统 的 Wintel 方案 可以 节省 至少 30 的 PCB 空间 按计划 今年 Q4 华硕 惠普 联想 将 首发 骁龙 835Win10 电脑 预计 均 是 二合一 形态 的 产品 当然 高通 骁龙 只是 个 开始 未来 也许 还 能 见到 三星 Exynos 联发科 华为 麒麟 小米 澎湃 等 进入 Windows10 桌面 平台',
 '此前 的 一加 3T 搭载 的 是 3400mAh 电池 DashCharge 快充 规格 为 5V4A 至于 电池 缩水 可能 与 刘作 虎 所说 一加 手机 5 要 做 市面 最 轻薄 大屏 旗舰 的 设定 有关 按照 目前 掌握 的 资料 一加 手机 5 拥有 55 寸 1080P 三星 AMOLED 显示屏 6G8GBRAM64GB128GBROM 双 1600 万 摄像头 备货 量 惊喜 根据 京东 泄露 的 信息 一加 5 起 售价 是 xx99 元 应该 是 在 279928992999 中 的 某个']

In [106]:
len(corpus),len(y)

(87054, 87054)

### 使用TF-IDF抽取文本特征

In [107]:
from sklearn.feature_extraction.text import TfidfVectorizer

- 调参经验：https://blog.csdn.net/blmoistawinde/article/details/80816179

- 除了max_features外，其他参数在以上链接均有说明
- max_features控制输出的特征数量，显然会输出一个稀疏矩阵！

In [112]:
TV = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b", max_df=0.6, stop_words=["是", "的"],max_features=100)
X = TV.fit_transform(corpus)

In [113]:
X.shape

(87054, 100)

### 分离训练集和测试集

In [111]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

- 随机划分数据集，或者分层划分数据集

In [117]:
train_set, test_set = train_test_split(X,test_size=0.2,random_state=42)

In [122]:
train_set.shape,test_set.shape

((69643, 100), (17411, 100))

- 由于样本分类不均衡，正负样本比例相对悬殊，必须对样本进行分层划分！

In [129]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(X,y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [130]:
strat_train_set.shape,strat_test_set.shape

((69643, 100), (17411, 100))

### 训练模型

- 用交叉验证对模型在训练集上训练，因为是分类模型，所以用分类模型的评价指标进行评价

In [125]:
from sklearn.model_selection import cross_val_score

In [144]:
def show_scores(model):
    p = cross_val_score(model,X_train,y_train,cv=5,scoring='precision')
    r = cross_val_score(model,X_train,y_train,cv=5,scoring='recall')
    f1 = cross_val_score(model,X_train,y_train,cv=5,scoring='f1')
    auc = cross_val_score(model,X_train,y_train,cv=5,scoring='roc_auc')
    print('precision is {}'.format(p))
    print('recall is {}'.format(r))
    print('f1 is {}'.format(f1))
    print('roc_auc is {}'.format(auc))

#### 随机分类器

In [161]:
from sklearn.linear_model import SGDClassifier
lr_clf = SGDClassifier(loss='log')

- 准确率：precision
- 所有被分为正例的样本中，真正的正例占多少比例
- $P = \frac{TP}{TP+FP}$

- 由于正例占比达到90%，precision在这里其实并不是一个好的衡量指标
- 因为即使分类器把所有样本分类为正例，其准确率也会高达90%！

In [162]:
p = cross_val_score(lr_clf,X_train,y_train,cv=5,scoring='precision')
print('precision ',p)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


precision  [0.97343677 0.97295819 0.97509624 0.96896149 0.97396895]


- 召回率：recall
- 所有真正是正例的样本中，被分为正例的比例
- $R = \frac{TP}{TP+FN}$

In [163]:
r = cross_val_score(lr_clf,X_train,y_train,cv=5,scoring='recall')
r

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


array([0.98720801, 0.99022724, 0.98665184, 0.98776418, 0.98712753])

- F1-Score：准确率和召回率的调和平均
- $F1 = \frac{2TP}{2TP+FP+FN}$

In [164]:
f = cross_val_score(lr_clf,X_train,y_train,cv=5,scoring='f1')
f

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


array([0.98018827, 0.98078286, 0.98034107, 0.97898582, 0.98024682])

- ROC：由TPR和FPR绘制而成
- Y轴$TPR = \frac{TP}{TP+FN}$
- X轴$TPR = \frac{TN}{TN+FP}$
- ROC_AUC：ROC曲线下的面积

- 在正负样本不均衡的情况下，AUC值更能反应分类器的性能高低！

In [165]:
a = cross_val_score(lr_clf,X_train,y_train,cv=5,scoring='roc_auc')
a

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=

array([0.98617757, 0.98487558, 0.9837698 , 0.98438253, 0.98597466])

#### KNN模型

In [137]:
from sklearn.neighbors import KNeighborsClassifier
knn_clf = KNeighborsClassifier()

- KNN模型训练的速度太慢！这里只看一下precision就够了。

In [138]:
a = cross_val_score(knn_clf,X_train,y_train,cv=5,scoring='precision')

In [145]:
a

array([0.96866841, 0.96792542, 0.96686961, 0.96704078, 0.96623274])

#### 朴素贝叶斯模型

In [146]:
from sklearn.naive_bayes import GaussianNB
gnb_clf = GaussianNB()

In [166]:
p = cross_val_score(gnb_clf,X_train.todense(),y_train,cv=5,scoring='precision')
r = cross_val_score(gnb_clf,X_train.todense(),y_train,cv=5,scoring='recall')
f1 = cross_val_score(gnb_clf,X_train.todense(),y_train,cv=5,scoring='f1')
auc = cross_val_score(gnb_clf,X_train.todense(),y_train,cv=5,scoring='roc_auc')
print('precision ',p)
print('recall ',r)
print('F1-score ',f1)
print('ROC-AUC ',auc)

precision  [0.99614315 0.99659773 0.99644128 0.99572154 0.9964241 ]
recall  [0.88240903 0.88439536 0.88987764 0.88757349 0.88565753]
F1-score  [0.93583316 0.93715007 0.94014942 0.93854232 0.93778133]
ROC-AUC  [0.96304718 0.96577782 0.96434328 0.96106288 0.96455802]


#### 逻辑回归模型

In [185]:
from sklearn.linear_model import LogisticRegression
lr1_clf = LogisticRegression(n_jobs  = -1,max_iter = 1000,verbose = 1,C=0.01,solver='newton-cg')

In [177]:
auc = cross_val_score(lr1_clf,X_train,y_train,cv=5,scoring='roc_auc')

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    8.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    3.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    3.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    2.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    2.2s finished


In [178]:
auc

array([0.96491152, 0.9653875 , 0.96469845, 0.96327178, 0.96567555])

#### XGBoost模型

- 集成方法非常容易过拟合！这里提前进行调参

In [173]:
import xgboost as xgb
xgb_clf = xgb.XGBClassifier(max_depth = 7, learning_rate = 0.1, sub_sample = 0.7,
                         n_estimators = 20, reg_alpha = 0.1, reg_lambda = 1,colsample_bytree = 0.7)

- 相当不错的成绩🥰

In [179]:
show_scores(xgb_clf)

precision is [0.99075403 0.98996096 0.98994012 0.98911316 0.99131405]
recall is [0.98760528 0.98720801 0.98514222 0.98895598 0.98847835]
f1 is [0.98917714 0.98858257 0.98753534 0.98903456 0.98989417]
roc_auc is [0.99520266 0.99478462 0.99358049 0.99510853 0.99486147]


#### LightGBM

In [181]:
import lightgbm as lgb
lgb_clf = lgb.LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.9,
        importance_type='split', learning_rate=0.1, max_depth=15)

- 强得可怕🥳

In [182]:
show_scores(lgb_clf)

precision is [0.99418605 0.99363311 0.99219124 0.99118978 0.99387577]
recall is [0.9918163  0.99197521 0.98935325 0.99221357 0.99292809]
f1 is [0.99299976 0.99280347 0.99077021 0.99170141 0.9934017 ]
roc_auc is [0.99793122 0.99776356 0.99689386 0.99761859 0.9976634 ]


### 利用测试集测试六个模型

In [186]:
a1 = cross_val_score(lr_clf,X_test,y_test,cv=3,scoring='roc_auc')
# a2 = cross_val_score(knn_clf,X_test,y_test,cv=3,scoring='roc_auc')  KNN速度太慢
a3 = cross_val_score(gnb_clf,X_test.todense(),y_test,cv=3,scoring='roc_auc')
a4 = cross_val_score(lr1_clf,X_test,y_test,cv=3,scoring='roc_auc')
a5 = cross_val_score(xgb_clf,X_test,y_test,cv=3,scoring='roc_auc')
a6 = cross_val_score(lgb_clf,X_test,y_test,cv=3,scoring='roc_auc')

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.3s finished


- 测试集上仍然是LightGBM最强

In [187]:
print('SGD: ',a1)
print('GNB: ',a3)
print('LR: ',a4)
print('XGBOOST: ',a5)
print('LIGHTGBM: ',a6)

SGD:  [0.94561301 0.94762092 0.9541019 ]
GNB:  [0.97001982 0.96936993 0.96185333]
LR:  [0.94561301 0.94762092 0.9541019 ]
XGBOOST:  [0.99427986 0.99339143 0.99241488]
LIGHTGBM:  [0.99646331 0.99551954 0.99508715]


### 对LightGBM进行调参

In [188]:
from sklearn.model_selection import GridSearchCV

- 暴力网格搜索确定参数

- 一共要训练60轮

In [197]:
params_grid = {'max_depth':  [5,10,15],
               'subsample': [0.75,0.9],
               'colsample_bytree': [0.75, 0.9],
               'learning_rate' : [0.1]}

In [198]:
lgb_new = lgb.LGBMClassifier()

In [199]:
lgb_gs = GridSearchCV(lgb_new,param_grid=params_grid,cv=5,scoring='roc_auc')

In [200]:
lgb_gs.fit(X_test,y_test)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'max_depth': [5, 10, 15], 'subsample': [0.75, 0.9], 'colsample_bytree': [0.75, 0.9], 'learning_rate': [0.1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

- 最佳参数

In [201]:
lgb_gs.best_params_

{'colsample_bytree': 0.9,
 'learning_rate': 0.1,
 'max_depth': 10,
 'subsample': 0.75}

In [202]:
lgb_gs.best_estimator_

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.9,
        importance_type='split', learning_rate=0.1, max_depth=10,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=0.75, subsample_for_bin=200000, subsample_freq=0)

In [206]:
cvres = lgb_gs.cv_results_

- 不断调整参数，auc达到99.64%差不多已经是极限了
- 极限可以理解为，不管再怎么调整参数，也无法超过的某个上限

In [208]:
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

0.9959062802741647 {'colsample_bytree': 0.75, 'learning_rate': 0.1, 'max_depth': 5, 'subsample': 0.75}
0.9959062802741647 {'colsample_bytree': 0.75, 'learning_rate': 0.1, 'max_depth': 5, 'subsample': 0.9}
0.9962282831229493 {'colsample_bytree': 0.75, 'learning_rate': 0.1, 'max_depth': 10, 'subsample': 0.75}
0.9962282831229493 {'colsample_bytree': 0.75, 'learning_rate': 0.1, 'max_depth': 10, 'subsample': 0.9}
0.9963978252184169 {'colsample_bytree': 0.75, 'learning_rate': 0.1, 'max_depth': 15, 'subsample': 0.75}
0.9963978252184169 {'colsample_bytree': 0.75, 'learning_rate': 0.1, 'max_depth': 15, 'subsample': 0.9}
0.9957992728419509 {'colsample_bytree': 0.9, 'learning_rate': 0.1, 'max_depth': 5, 'subsample': 0.75}
0.9957992728419509 {'colsample_bytree': 0.9, 'learning_rate': 0.1, 'max_depth': 5, 'subsample': 0.9}
0.9964063009970281 {'colsample_bytree': 0.9, 'learning_rate': 0.1, 'max_depth': 10, 'subsample': 0.75}
0.9964063009970281 {'colsample_bytree': 0.9, 'learning_rate': 0.1, 'max_dep

- 确定最终模型

In [209]:
final_model = lgb_gs.best_estimator_

In [211]:
final_model.fit(X_test,y_test)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.9,
        importance_type='split', learning_rate=0.1, max_depth=10,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=0.75, subsample_for_bin=200000, subsample_freq=0)

- 在测试集上表现很不错
- 让人满意的结果

In [213]:
cross_val_score(final_model,X_test,y_test,cv=10,scoring='roc_auc')

array([0.99815832, 0.99870288, 0.99389436, 0.99647512, 0.99603427,
       0.99788091, 0.995005  , 0.99711652, 0.9930524 , 0.99743615])

### 寻找抄袭者

In [214]:
pred = final_model.predict(X)

In [216]:
data.head()

,content,y
0,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,0.0
1,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,0.0
2,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,0.0
3,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,1.0
4,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,0.0


In [226]:
data['source'] = new_df['source']

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [227]:
data['y_pred'] = pred

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


- 把模型对文章的预测，和文章的来源并入DataFrame

In [228]:
data.head()

,content,y,y_pred,source
0,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,0.0,0.0,快科技@http://www.kkj.cn/
1,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,0.0,0.0,快科技@http://www.kkj.cn/
2,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,0.0,0.0,快科技@http://www.kkj.cn/
3,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,1.0,1.0,新华社
4,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,0.0,0.0,深圳大件事


- plagarism视为剽窃者

In [229]:
plag = data[(data['y']==0) & (data['y_pred']==1)]

In [230]:
plag.head()

,content,y,y_pred,source
15,中国证券网讯（记者 费天元）沪深两市周五临近午盘出现快速跳水，沪指连续跌穿5日、10日及...,0.0,1.0,中国证券网
37,切尔西球星法布雷加斯正与女友达涅拉-塞曼享受假期，在个人的INSTAGRAM上，小法发布...,0.0,1.0,新浪体育
44,从本质上说，苹果希望改进当前的一种抓取光体图的方法，这种方法可以很有效地测量这些体积的变化。...,0.0,1.0,TechWeb@
45,在天山北麓，辽阔的伊犁大草原上，水草丰茂，土地肥沃，自古就出骏马。天山出天马，新疆伊犁哈萨克...,0.0,1.0,人民网
51,戈壁的大漠黄沙曾掩埋了无数西域古道，而如今一条大漠天路正顽强地与黄沙“搏斗”，在乌兰布和、腾...,0.0,1.0,新华网


- 共有517篇文章涉嫌抄袭

In [231]:
plag.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 517 entries, 15 to 10970
Data columns (total 4 columns):
content    517 non-null object
y          517 non-null float64
y_pred     517 non-null float64
source     517 non-null object
dtypes: float64(2), object(2)
memory usage: 20.2+ KB


### 什么是数据思维？什么是机器学习思维？

- 数据思维：

我认为数据思维，其实就是量化的方法。我们把要达到的目标和要做的事情，全部分解成可以用数学语言或量化数据去描述的特征。例如，如果问一个班里面，谁是最优秀的学生？我们可以用德智体美劳加权成绩来衡量一个学生的优劣，成绩最高为优，最低为劣。
- 机器学习思维：

我认为机器学习思维，其实就是数据思维的升级版。机器学习思维，是基于量化目标的前提下，找到一个可以优化的目标函数，或者是损失函数，一步一步向目标进化，缩小误差，这就是机器学习思维。

### 选做之基于编辑距离的文本相似度

- 编辑距离是一种可以用查看字符串之间‘相差多远’的算法
- 基于编辑距离可以计算出两个文本之间的相似程度
- 由于已经有现成的库，在此就不自己再造轮子了

- [Levenshtein函数说明](https://rawgit.com/ztane/python-Levenshtein/master/docs/Levenshtein.html)

In [2]:
import Levenshtein as le

In [5]:
a = 'I love you'
b = 'I like you'

In [6]:
le.ratio(a,b)

0.8

- 怎么寻找抄了新华社哪篇文章？